In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [3]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

In [4]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('E:/Mini project/Dataset/new/Audio_Song_Actors_01-24/Actor_*/*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [5]:
# Split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [6]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1839, 613)


In [7]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [8]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [9]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [10]:
# Predict for the test set
y_pred=model.predict(x_test)
#y_pre = model.predict([feature])
#print(y_pre)

In [11]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))


Accuracy: 65.25%


In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

       angry       0.79      0.79      0.79        84
        calm       0.62      0.84      0.71        94
     disgust       0.76      0.35      0.48        37
     fearful       0.68      0.68      0.68       103
       happy       0.76      0.70      0.73        99
     neutral       0.51      0.78      0.62        46
         sad       0.90      0.27      0.42        99
   surprised       0.44      0.78      0.57        51

    accuracy                           0.65       613
   macro avg       0.68      0.65      0.62       613
weighted avg       0.71      0.65      0.64       613



In [13]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[66  2  1  3  2  2  1  7]
 [ 0 79  2  1  1  9  0  2]
 [ 6  2 13  3  1  2  0 10]
 [ 4  7  1 70  7  4  2  8]
 [ 5  5  0  8 69  3  0  9]
 [ 0  5  0  1  1 36  0  3]
 [ 3 23  0 16  5 14 27 11]
 [ 0  4  0  1  5  1  0 40]]
